In [1]:
import torch
import numpy as np
import data
import tqdm
import models
from torchvision.transforms import v2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

/home/anirudh/Documents/GSoC/DeepLense/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda


In [ ]:
def transform(image):
    """
    Data augmentation by random rotation, and random scaling between 0.8 to 1.2

    :param image: Image to be transformed
    :return: Transformed image
    """
    return v2.RandomAffine(degrees=(-180,180),scale=(0.8,1.2))(image)

In [2]:
EPOCHS = 200
BATCH_SIZE = 15
LR = 1e-4
OUT_CLASSES = 3

In [3]:
train_dataset = data.LensingDataset('index.csv','../dataset/train/',transform=transform)
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [0.9, 0.1])

train_dataloader = torch.utils.data.DataLoader(train_dataset,shuffle=True,batch_size=BATCH_SIZE,num_workers=15)
val_dataloader = torch.utils.data.DataLoader(val_dataset,shuffle=True,batch_size=BATCH_SIZE,num_workers=15)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self, image_channels=1, num_classes=3) -> None:
        super(Model, self).__init__()
        self.model = models.ResNet18(image_channels=image_channels,num_classes=num_classes).to(device)
    
    def forward(self, x):
        x = self.model(x)
        return x

In [5]:
model = Model().to(device)


loss_fn = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(params=model.parameters(),lr=LR)
history = {'val_loss':[], 'acc':[], 'loss':[]}
best_val_loss = np.inf
best_weights = None
for epoch in range(EPOCHS):
    losses = []
    accs = []
    model.train()
    for i,(x,y) in enumerate(tqdm.tqdm(train_dataloader, desc=f"Training epoch {epoch+1}")):
        (x, y) = (x.float().to(device), y.to(device))
        x = model(x)
        loss = loss_fn(x,y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        label_indices = torch.argmax(y, 1)
        prediction_indices = torch.argmax(x, 1)
        acc = float(torch.sum(label_indices == prediction_indices))/float(len(label_indices))
        losses.append(float(loss))
        accs.append(float(acc))
    print("Training at epoch %d/%d: %.4f with acc=%.5f at LR=%.6f"%(epoch+1,EPOCHS,np.mean(losses),np.mean(accs),opt.param_groups[0]['lr']))
    history['loss'].append(np.mean(losses))
    model.eval()
    losses = []
    accs = []
    for i,(x,y) in enumerate(tqdm.tqdm(val_dataloader, desc=f"Validation epoch {epoch}")):
        (x, y) = (x.float().to(device), y.to(device))
        x = model(x)
        val_loss = loss_fn(x,y)
        label_indices = torch.argmax(y, 1)
        prediction_indices = torch.argmax(x, 1)
        acc = float(torch.sum(label_indices == prediction_indices))/float(len(label_indices))
        losses.append(float(val_loss))
        accs.append(float(acc))
    if best_val_loss > np.mean(losses):
        best_val_loss = np.mean(losses)
        best_weights = model.state_dict()
    history['val_loss'].append(np.mean(losses))
    history['acc'].append(np.mean(accs))
    print(f"Validation at epoch {epoch+1}/{EPOCHS}: {np.mean(losses)} with acc={np.mean(accs)} at LR={opt.param_groups[0]['lr']}")



Training epoch 1: 100%|██████████| 1800/1800 [03:01<00:00,  9.89it/s]


Training at epoch 1/200: 1.1370 with acc=0.33159 at LR=0.000100


Validation epoch 0: 100%|██████████| 200/200 [00:08<00:00, 24.67it/s]


Validation at epoch 1/200: 1.131750868856907 with acc=0.3456666666666666 at LR=0.0001


Training epoch 2: 100%|██████████| 1800/1800 [03:00<00:00,  9.99it/s]


...

Validation epoch 137: 100%|██████████| 200/200 [00:07<00:00, 25.08it/s]


Validation at epoch 138/200: 0.7085883703827858 with acc=0.696 at LR=0.0001


Training epoch 139: 100%|██████████| 1800/1800 [02:59<00:00, 10.02it/s]


Training at epoch 139/200: 0.6100 with acc=0.72733 at LR=0.000100


Validation epoch 138:  58%|█████▊    | 116/200 [00:05<00:02, 29.83it/s]

In [ ]:
model.load_state_dict(best_weights)
torch.save(model.state_dict(), '../trained_weights/resnet_18.pt')